# Imports and setting up viz

NB : conda env1 on Mac, lam1env on spirit (Python3.12)

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *

In [2]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load files

LAM output analysis.

Simu irr et simu no-irr.

## Sims

In [3]:
noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/noirr_2010_2022'
irr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/irr_2010_2022'

In [ ]:
#open netcdf files for native routing on ORC grid
# filename = '{}/SRF/MO/*history.nc'.format(noirr_dir)
filename = '{}/SRF/TS_MO_noirr*.nc'.format(noirr_dir)

ORCnoirr0 = xr.open_mfdataset(filename)
ORCnoirr = ORCnoirr0.rename({'time_counter':'time'})
ORCnoirr.attrs['name'] = 'no_irr'
ORCnoirr = ORCnoirr.sel(lon=slice(-13,6),lat=slice(32,49))
ORCnoirr

In [ ]:
# filename = '{}/SRF/MO/*history.nc'.format(irr_dir)
filename = '{}/SRF/TS_MO_irr*.nc'.format(irr_dir)

ORCirr0 = xr.open_mfdataset(filename)
ORCirr = ORCirr0.rename({'time_counter':'time'})
ORCirr.attrs['name'] = 'irr'
ORCirr = ORCirr.sel(lon=slice(-13,6),lat=slice(32,49))

# #manually define irrig_deficit as netirrig-irrigation in ORC file
# ORCirr['irrig_deficit'] = ORCirr['netirrig'] - ORCirr['irrigation']
# #make irrig_deficit units mm/day
# ORCirr['irrig_deficit'].attrs['units'] = 'mm/day'
# ORCirr['irrig_frac'] = ORCirr['irrigmap_dyn']/ORCirr['Areas']

ORCirr0

## Period selection

In [6]:
#select only JJA
ORCnoirr = ORCnoirr.sel(time=ORCnoirr['time.season']=='JJA')
ORCirr = ORCirr.sel(time=ORCirr['time.season']=='JJA')
# newORC = newORC.sel(time=newORC['time.season']=='JJA')

## Masks

In [7]:
#create a masks of data points where irrig_frac >5%
irr_mask = (ORCirr['irrigmap_dyn']/ORCirr['Areas'])>0.05

In [8]:
mORCirr=ORCirr.where(irr_mask)
mORCirr.attrs['name'] = 'irr'

mORCnoirr=ORCnoirr.where(irr_mask)
mORCnoirr.attrs['name'] = 'no_irr'

In [9]:
con_mask=ORCnoirr['Contfrac']>0.9
ip_mask=polygon_to_mask(ORCnoirr, iberic_peninsula)

In [10]:
ip_ORCnoirr=ORCnoirr.where(con_mask).where(ip_mask['mask'])
ip_ORCirr=ORCirr.where(con_mask).where(ip_mask['mask'])

# Maps

In [ ]:
ds=ORCirr
var='irrigation'
# plotvar=ds[var].mean(dim='time') * 100
# map_plotvar(plotvar, cmap=reds, hex=False, vmax=80, title='Irrigated fraction (%)')
vmax=1.5
vmin=None
title='Irrigation (mm/day, JJA)'
map_ave(ds, var, vmin=vmin, vmax=vmax, title=title, cmap=wet, hex=False)

In [ ]:
#diff
ds1=ip_ORCirr
ds2=ip_ORCnoirr
var='rain'
vmax=15
vmin=-vmax
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax)

In [ ]:
#diff with significance
ds1=ip_ORCirr   
ds2=ip_ORCnoirr
var='streamr'
vmax=45
vmin=-vmax
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, sig=True)

In [ ]:
#relative difference
ds1=ip_ORCirr
ds2=ip_ORCnoirr
var='streamr'
vmax=50
vmin=-50
map_rel_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax)

In [ ]:
#Display map of var for ds on restricted area
var='irrigation'
ds = ORCirr
latmin=41.3
latmax=41.9
lonmin= 0.6
lonmax= 1.4
# latmin=41.
# latmax=42.5
# lonmin= 0.
# lonmax= 1.5

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')
map_plotvar(plotvar, cmap=wet, vmin=0, vmax=1.4, title='JJA applied irrigation (mm/day)')

# Time series

In [ ]:
var='snowf'

ds1=ip_ORCnoirr
ds2=ip_ORCirr
ds_list=[ds1, ds2]
# ds_list=[ds2]

year_max=2022
time_series_ave(ds_list, var, year_max=year_max, title=var)
seasonal_cycle_ave(ds_list, var, title=var)

In [ ]:
seasonal_cycle_ave(ds_list, 'precisol', title=var)

In [ ]:
seasonal_cycle_ave(ds_list, 'snow', title=var)